In [39]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


def fetch_with_web_unlocker(url, proxy_username, proxy_password, proxy_url):
    proxies = {
        'http': f'http://{proxy_username}:{proxy_password}@{proxy_url}',
        'https': f'http://{proxy_username}:{proxy_password}@{proxy_url}'
    }

    response = requests.get(url, proxies=proxies, verify=False)

    if response.status_code == 200:
        return response.text
    else:
        return "Failed to retrieve the page"

# Define sensitive data

proxy_username = os.getenv('proxy_username')
proxy_password = os.getenv('proxy_password')
proxy_url = os.getenv('proxy_url')

# URL of the page to scrape
url = "https://ra.co/events/us/sanfrancisco"

# Fetch and print the HTML of the page using Web Unlocker
html_content = fetch_with_web_unlocker(url, proxy_username, proxy_password, proxy_url)
print(html_content)

/Users/maxhager/.virtualenvs/base/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<!doctype html>
<html lang="en" class="js-focus-visible" data-js-focus-visible=""><head style=""><style type="text/css"></style><meta charset="utf-8"><meta name="twitter:card" content="summary_large_image"><meta name="twitter:site" content="@residentadvisor"><meta property="fb:app_id" content="712773712080127"><meta property="og:image" content="https://public.ra.co/images/ra-co-logo.jpeg"><meta property="og:site_name" content="Resident Advisor"><meta name="viewport" content="width=device-width, initial-scale=1.0"><link rel="preload" href="/static/AlternateGothicProCyrillic.woff2" as="font" crossorigin="anonymous" type="font/woff2"><link rel="preload" href="/static/RobotoMono-Regular.woff2" as="font" crossorigin="anonymous" type="font/woff2"><link rel="alternate" hreflang="de" href="https://de.ra.co/events/us/sanfrancisco" data-testid="alternate-link-german"><link rel="alternate" hreflang="es" href="https://es.ra.co/events/us/sanfrancisco" data-testid="alternate-link-spanish"><link rel=

In [44]:
from bs4 import BeautifulSoup

# Sample HTML content
# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extract the first section
first_section = soup.find('div', class_='Box-sc-abq4qd-0 esoJCv')

print(first_section)

<div class="Box-sc-abq4qd-0 esoJCv"><div class="Box-sc-abq4qd-0 Sticky__SectionStyledBox-sc-17cws0i-0 fyQlU" data-testid="sticky" style="position: sticky; top: 0px;"><h3 class="Box-sc-abq4qd-0 Heading__StyledBox-rnlmr6-0 duPGEf" data-testid="grouped-header-heading"><span class="Text-sc-wks9sf-0 faOCUz" color="accent" font-weight="normal"><span class="Text-sc-wks9sf-0 iMtJFl" color="accent" font-weight="normal"≯</span>Wed, 19 Jun</span></h3></div><div class="Box-sc-abq4qd-0 dLKsgz" data-pw-test-id="event-listing-item-ticketed" data-testid="event-listing-card"><hr class="Divider__HorizontalDivider-sc-olo2rq-0 joZAFY"/><ul class="Grid__GridStyled-sc-si5izk-0 ehphoN grid" data-test-id="ticketed-event"><li class="Column-sc-4kt5ql-0 gmoGYX"><a class="Link__AnchorWrapper-sc-1huefnz-1 kQSgWi" data-pw-test-id="event-image-link" data-tracking-id="https://ra.co/events/1872540" href="https://ra.co/events/1872540"><span class="Text-sc-wks9sf-0 Link__StyledLink-sc-1huefnz-0 BfwyQ" color="primary" d

In [46]:
import re

event_links = first_section.find_all('a', href=re.compile(r'https://ra.co/events/\d+'))

# Extract the href attributes to get the links
unique_links = set()
for link in event_links:
    if 'href' in link.attrs:
        unique_links.add(link['href'])

print(list(unique_links))

['https://ra.co/events/1872540', 'https://ra.co/events/1947577', 'https://ra.co/events/1948804', 'https://ra.co/events/1948788']


In [ ]:
#i have all the events next i need to get the name and the address 
urls=['https://ra.co/events/1872540', 'https://ra.co/events/1947577', 'https://ra.co/events/1948804', 'https://ra.co/events/1948788']


In [66]:
#once i have this addresses together i can save it to the DB
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define sensitive data
proxy_username = os.getenv('proxy_username')
proxy_password = os.getenv('proxy_password')
proxy_url = os.getenv('proxy_url')

# Function to fetch HTML content using proxies with retry mechanism
def fetch_with_web_unlocker(url, proxy_username, proxy_password, proxy_url, retries=3):
    proxies = {
        'http': f'http://{proxy_username}:{proxy_password}@{proxy_url}',
        'https': f'http://{proxy_username}:{proxy_password}@{proxy_url}'
    }
    attempt = 0
    while attempt < retries:
        response = requests.get(url, proxies=proxies, verify=False)
        if response.status_code == 200:
            return response.text
        attempt += 1
    return None  # Return None if all retries fail

# URLs of the event pages
urls = [
    'https://ra.co/events/1872540', 
    'https://ra.co/events/1947577', 
    'https://ra.co/events/1948804', 
    'https://ra.co/events/1948788'
]

# Loop through each URL, fetch the content, and parse for name and address
# Loop through each URL, fetch the content, and parse for name and address
event_details = []
for url in urls:
    html_content = fetch_with_web_unlocker(url, proxy_username, proxy_password, proxy_url)
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        name = soup.find('h1', class_='Heading__StyledBox-rnlmr6-0').text.strip()
        # Find the address within the specific span element
        address_container = soup.find('li', class_='Column-sc-4kt5ql-0')
        if address_container:
            address_span = address_container.find('span', class_='Text-sc-wks9sf-0', string=lambda text: "St" in text or "Ave" in text or "Blvd" in text or "Rd" in text)
            address = address_span.text.strip() if address_span else "Address not found"
        else:
            address = "Address container not found"
        event_details.append({'url': url, 'name': name, 'address': address})
        print("event details for ", name, "with address ", address, "and url ", url)
    else:
        print(f"Failed to fetch content for {url}.", html_content)

# Print the extracted details
for event in event_details:
    print(event)

/Users/maxhager/.virtualenvs/base/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


event details for  Black Techno Matters | Liber8 SF with address  1192 Folsom St, San Francisco, CA 94103 and url  https://ra.co/events/1872540


/Users/maxhager/.virtualenvs/base/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


event details for  DJ Caro, Wockie with address  3152 Mission Street San Francisco, CA 94110 and url  https://ra.co/events/1947577


/Users/maxhager/.virtualenvs/base/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


event details for  SHCKLDBYLUST Happy Hour with address  1192 Folsom St, San Francisco, CA 94103 and url  https://ra.co/events/1948804


/Users/maxhager/.virtualenvs/base/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


event details for  Crash Course presents Watts Black Panda RCA with address  1192 Folsom St, San Francisco, CA 94103 and url  https://ra.co/events/1948788
{'url': 'https://ra.co/events/1872540', 'name': 'Black Techno Matters | Liber8 SF', 'address': '1192 Folsom St, San Francisco, CA 94103'}
{'url': 'https://ra.co/events/1947577', 'name': 'DJ Caro, Wockie', 'address': '3152 Mission Street San Francisco, CA 94110'}
{'url': 'https://ra.co/events/1948804', 'name': 'SHCKLDBYLUST Happy Hour', 'address': '1192 Folsom St, San Francisco, CA 94103'}
{'url': 'https://ra.co/events/1948788', 'name': 'Crash Course presents Watts Black Panda RCA', 'address': '1192 Folsom St, San Francisco, CA 94103'}


In [61]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Supabase setup
url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_KEY')
supabase: Client = create_client(url, key)

# Example event details
# event_details = [
#     {'url': 'https://ra.co/events/1872540', 'name': 'Black Techno Matters | Liber8 SF', 'address': 'VenueF8 1192 Folsom1192 Folsom St, San Francisco, CA 94103'},
#     {'url': 'https://ra.co/events/1947577', 'name': 'DJ Caro, Wockie', 'address': 'VenueMothership3152 Mission Street San Francisco, CA 94110'},
#     {'url': 'https://ra.co/events/1948804', 'name': 'SHCKLDBYLUST Happy Hour', 'address': 'VenueF8 1192 Folsom1192 Folsom St, San Francisco, CA 94103'},
#     {'url': 'https://ra.co/events/1948788', 'name': 'Crash Course presents Watts Black Panda RCA', 'address': 'VenueF8 1192 Folsom1192 Folsom St, San Francisco, CA 94103'}
# ]

# Prepare data for single row insertion
data = {
    'events': event_details  # Storing all events in a single key as a list
}

# Insert data into the database
response = supabase.table('resident_advisor').insert(data).execute()

2024-06-20 14:34:24,987:INFO - HTTP Request: POST https://aisbbafpdsjkbpqmdhty.supabase.co/rest/v1/resident_advisor "HTTP/1.1 201 Created"


AttributeError: 'APIResponse[~_ReturnT]' object has no attribute 'status_code'